# How to use the Molcube PDB Reader API

### Part 0: Package installation
---

####  Please use the follwing command to install the package:

In [ ]:
pip install -U http://deployments.molcube.com/api/molcube-0.3.1-py3-none-any.whl

### Part 1: Authentication and API Setup
---

In [15]:
import molcube as mc
import os

# Connect to the MolCube API server.
molcube = mc.API('api.molcube.com', 443)

# Enter the username and password for API authentication.
# (For security, it is recommended to store credentials in a separate file)
# e.g., using: %run keys.py

username = "your_actual_username"
password = "your_actual_password"

%run keys.ipynb # This is my personal keys
molcube.authenticate(username=username, password=password)

print("✅ MolCube API Authentication Successful!")

# Create a directory to store the results.
output_dir = './molcube_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

✅ MolCube API Authentication Successful!


### Part 2-1: Using PDB from RCSB Database
---

In [ ]:
# Set a title for the job.
title_rcsb = 'RSCB_PDB_test'

# Initialize the PDBReader project object.
pdbreader_rcsb = molcube.create_pdb_reader_project()

# Create a new project with 'charmmff' as the default force field.
# [Available Force Fields: charmmff, martiniff, amberff, drudeff]
# Read the PDB ID '6x1a' from the RCSB database.
# Additional options like correct_topo, rename_dupl_atoms, and calc_pka can be used for structure correction.
pdbreader_rcsb.create_project(title=title_rcsb, ff='charmmff', pdb_id='6x1a')
print("✅ Loaded PDB '6x1a' from RCSB.")

# Analyze and retrieve ligand information from the PDB file.
available_info_rcsb, _ = pdbreader_rcsb.available_info()
available_ligand_rcsb = available_info_rcsb.get('standaloneLigand', [])

# Set the options for ligand processing.
chain_options, option_rcsb = pdbreader_rcsb.model_options()
standalone_ligands = []
for lig_info in available_ligand_rcsb:
    standalone_ligands.append(
        {
            'resname': lig_info.get('resname'),      # The residue name of the ligand to process.
            'chainIndex': lig_info.get('chainIndex'),# The chain index where the ligand is located.
            'ffType': "openff",                      # Force field to apply to the ligand. # [openff, gaff, xff]
            'fast': True                             # Whether to use the fast calculation mode. # [True, False]
        }
    )
print("✅ Set modeling options for the ligand.")

# Select chains and ligands for modeling.
pdbreader_rcsb.select_chains(chain_options, standalone_ligands)

# Execute the PDB modeling based on the configured options.
pdbreader_rcsb.model_pdb(option=option_rcsb)
print("✅ PDB modeling complete.")

# Download the processed PDB file to the specified path.
output_path_rcsb = os.path.join(output_dir, 'RCSB_6x1a.pdb')
pdbreader_rcsb.download_pdb(output_path_rcsb)
print(f"👍 Results saved to '{output_path_rcsb}'.")

task finished...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:36<00:00,  7.25s/it]


✅ Loaded PDB '6x1a' from RCSB.

[PROTEIN]
  - Chain Index: PROT_A
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_B
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_C
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_D
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_E
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE

[STANDALONELIGAND]
  - Chain Index: HETE_A
    Residue Name: UK4
✅ Set modeling options for the ligand.


optimize missing residue...:  75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                          | 9/12 [01:28<00:29,  9.82s/it]

### Part 2-2: Using a custom PDB File
---

In [ ]:
from molcube.pdbreader.enum import PDBSource

# Set a title for the new job.
title_local = 'Custom_PDB_test'

# Initialize a new PDBReader project object.
pdbreader_local = molcube.create_pdb_reader_project()

# Create the project with 'charmmff' as the default force field.
# [Available Force Fields: charmmff, martiniff, amberff, drudeff]
# Upload a PDB file from your local machine to the server.
# (This path must be changed to match your actual file location)
local_pdb_path = "/Users/sangmin/Downloads/6x1a.pdb"
pdbreader_local.create_project(title=title_local, ff='charmmff', pdb_source=PDBSource.CUSTOM, pdb_path=local_pdb_path)
print(f"✅ Uploaded local PDB '{local_pdb_path}'.")

# Analyze ligand information from the uploaded file.
available_info_local, _ = pdbreader_local.available_info()
available_ligand_local = available_info_local.get('standaloneLigand', [])

# Set the options for ligand processing.
chain_options, option_local = pdbreader_local.model_options()
standalone_ligands = []
for lig_info in available_ligand_local:
    standalone_ligands.append(
        {
            'resname': lig_info.get('resname'),
            'chainIndex': lig_info.get('chainIndex'),
            'ffType': "gaff",                        # Using 'gaff' force field this time. # [openff, gaff, xff]
            'fast': True                             # [True, False]
        }
    )
print("✅ Set modeling options for the ligand.")

# Select chains and ligands for modeling.
pdbreader_local.select_chains(chain_options, standalone_ligands)

# Execute the PDB modeling.
pdbreader_local.model_pdb(option=option_local)
print("✅ PDB modeling complete.")

# Download the final processed PDB file.
output_path_local = os.path.join(output_dir, 'custom_6x1a.pdb')
pdbreader_local.download_pdb(output_path_local)
print(f"👍 Results saved to '{output_path_local}'.")

task finished...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:28<00:00,  5.66s/it]


✅ Uploaded local PDB '/Users/sangmin/Downloads/6x1a.pdb'.

[PROTEIN]
  - Chain Index: PROT_A
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_B
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_C
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_D
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE
  - Chain Index: PROT_E
    Terminal:
      NTER: NTER, NNEU, ACE, NONE
      CTER: CTER, CNEU, CT1, CT2, CT3, NONE

[STANDALONELIGAND]
  - Chain Index: HETE_A
    Residue Name: UK4
✅ Set modeling options for the ligand.


save system...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [09:32<00:00, 47.73s/it]


✅ PDB modeling complete.
👍 Results saved to './molcube_results/custom_6x1a.pdb'.
